In [1]:
import d2lzh as d2l
import mxnet as mx
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import loss as gloss,nn
import time

In [2]:
net = nn.Sequential()

In [4]:
net.add(nn.Conv2D(channels=6,kernel_size=5,activation='sigmoid'),
               nn.MaxPool2D(pool_size=2,strides=2),
               nn.Conv2D(channels=16,kernel_size=5,activation='sigmoid'),
               nn.MaxPool2D(pool_size=2,strides=2),
               nn.Dense(120,activation='sigmoid'),
               nn.Dense(84,activation='sigmoid'),
               nn.Dense(10))

In [5]:
x=nd.random.uniform(shape=(1,1,28,28))
net.initialize()
for layer in net:
    x=layer(x)
    print(layer.name, 'output shape:\t',x.shape)

conv1 output shape:	 (1, 6, 24, 24)
pool1 output shape:	 (1, 6, 12, 12)
conv2 output shape:	 (1, 16, 8, 8)
pool2 output shape:	 (1, 16, 4, 4)
dense0 output shape:	 (1, 120)
dense1 output shape:	 (1, 84)
dense2 output shape:	 (1, 10)


In [6]:
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size=batch_size)

In [27]:
def evaluate_acc(data_iter,net):
    acc_sum,n=0.0,0
    for x,y in data_iter:
        y=y.astype('float32')
        acc_sum+=(net(x).argmax(axis=1)==y).sum()
        n+=y.size
    return acc_sum.asscalar()/n

In [31]:
def train_ch5(net,train_iter,test_iter,batch_szie,trainer,num_epochs):
    loss=gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum,train_acc_sum,n,start=0.0,0.0,0,time.time()
        for x,y in train_iter:
            with autograd.record():
                y_hat=net(x)
                l=loss(y_hat,y).sum()
            l.backward()
            trainer.step(batch_size)
            y=y.astype('float32')
            train_l_sum+=l.asscalar()
            train_acc_sum+=(y_hat.argmax(axis=1)==y).sum().asscalar()
            n+=y.size
        test_acc=evaluate_acc(test_iter,net)
        print('epoch %d,loss %.4f,train_acc %.3f,test_acc %.3f,''time %.1f sec'
             %(epoch+1,train_l_sum/n,train_acc_sum/n,test_acc,time.time()-start))

In [32]:
lr,num_epochs=0.9,5
#net.initialize(force_reinit=True,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
train_ch5(net,train_iter,test_iter,batch_size,trainer,num_epochs)

epoch 1,loss 0.8357,train_acc 0.680,test_acc 0.717,time 31.9 sec
epoch 2,loss 0.7073,train_acc 0.722,test_acc 0.742,time 32.6 sec
epoch 3,loss 0.6459,train_acc 0.746,test_acc 0.770,time 34.0 sec
epoch 4,loss 0.5866,train_acc 0.770,test_acc 0.788,time 32.2 sec
epoch 5,loss 0.5456,train_acc 0.788,test_acc 0.809,time 32.0 sec
